# Journey Visualisations 

In [1]:
# Import Python Libraries 

import sys
import os
import pickle
import time 

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


from pandasql import sqldf
import sqlalchemy
import snowflake.connector

import pyodbc

from tqdm import tqdm

# import Utilities
# import importlib
# importlib.reload(Utilities)

import re
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

from datetime import timedelta


# show all dataframe columns 

pd.set_option('display.max_columns', None)

import statsmodels.api as sm
from statsmodels.formula.api import ols


from matplotlib.colors import to_rgba

plt.style.use('./CEIH.mplstyle')
color_dict = {0: to_rgba('#32C0D2', 1),
                1: to_rgba("#E0B165", 1),}

cmap_blended = sns.blend_palette(["#ADE6ED","#70D3E0","#32C0D2","#289AA8","#307078"], as_cmap=True)

from statsmodels.stats.multicomp import pairwise_tukeyhsd
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.formula.api as smf 

import plotly.express as px 
import plotly.io as pio

In [2]:
con = snowflake.connector.connect(
        account='wt61814.australia-east.privatelink',
        user="iain.bertram@sa.gov.au",
        database="DEV_DAP_CAE05_DB",
        warehouse="DEV_DAP_CAE05_M_WH",
        authenticator="externalbrowser",
        )

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://login.microsoftonline.com/bda528f7-fca9-432f-bc98-bd7e90d40906/saml2?SAMLRequest=pZNRb9owFIX%2FSuQ9x3FCCsQiVBRaxtR1qISp2ptJHLBw7NTXIe2%2FnxNA6h7al71Z9rn%2Bzr3Hnty%2BVdI7cQNCqxSFmCCPq1wXQu1TtM0e%2FDHywDJVMKkVT9E7B3Q7nQCrZE1njT2oZ%2F7acLCeu0gB7Q5S1BhFNQMBVLGKA7U53cx%2BPtIIE8oAuLEOhy4lBQjHOlhb0yBo2xa3A6zNPogIIQFJAqfqJN%2FQB0T9NaM22upcy2vJm%2BvpE0QYkLhDOIUjrC%2BFd0KdR%2FAVZXcWAf2eZWt%2F%2FWuTIW927W6uFTQVNxtuTiLn2%2BfHswHoHNhhOA5jzBqwhknBfM7A4tqIE7NcCnXEoHRbSnbkua7qxjoMdqug5EUg9V644a0WKaqPotjeJ3fxUu%2FKH0vWLpeLerHcvrzsk%2FFx9jRvDq9seCL3GePRcZUj7%2Fc16qiLegXQ8JXqArZui0Q3Phn5A5KRIR0MKQnxYBT%2FQd7CBSwUs33ltYveB65EbjTo0mrljPPe5a5gN9G4HPllzhI%2FHkSlv8uTsb8rRjwhRUwSMgy6GCN0fkq0N2Km%2Fz2gSfDxusszfXLJrRZrLUX%2B7j1oUzH7ebAhDvsdUfhlL6W8YkLOisJwABew

In [3]:
Data_Storage_File = 'Data\ED_Presentations_{}_{}_Version_01.pkl'.format(2023,2024)
print(Data_Storage_File)


with open(Data_Storage_File, 'rb') as file:
    sepsis_df = pd.read_pickle(file)



Data\ED_Presentations_2023_2024_Version_01.pkl


In [36]:
sepsis_2023_df = sepsis_df.loc[sepsis_df.ADMITDTM.between('2024-01-01','2025-01-01')].copy()


In [37]:
sepsis_positive_df = sepsis_2023_df.loc[sepsis_2023_df.ANY_SEPSIS==1].copy()

In [38]:
jids = tuple(sepsis_positive_df.JOURNEY_ID.unique())

In [39]:
jids

('41900903434741464538-101',
 '36870901634741464539-110',
 '15199191454741464540-106',
 '27628581454741464540-103',
 '65415903434741464538-106',
 '53330701634741464539-109',
 '21886201634741464539-105',
 '57767201634741464539-101',
 '25694011454741464540-102',
 '33839401634741464539-102',
 '45545401634741464539-123',
 '58179201634741464539-118',
 '53609301634741464539-104',
 '27896903434741464538-102',
 '40741801634741464539-122',
 '17469951454741464540-101',
 '64473901634741464539-101',
 '17059241454741464540-106',
 '97445901634741464539-111',
 '16461971454741464540-101',
 '59665601634741464539-108',
 '76904701634741464539-104',
 '13741351454741464540-107',
 '30115821454741464540-102',
 '44703301634741464539-109',
 '55516301634741464539-103',
 '27687601454741464540-103',
 '53401901634741464539-108',
 '81232201634741464539-108',
 '51704601634741464539-111',
 '33145903434741464538-107',
 '36140601634741464539-111',
 '25014701634741464539-496',
 '40157903434741464538-101',
 '406132016347

In [40]:
query2 = ''' 
select J.*,cod.drgcode from DEV_DAP_CAE05_DB.JNY.TB_PATIENT_JOURNEY_MAPPING_NEW as J
LEFT JOIN PRD_DAP_EMR_TFM_DB.EMR_TFM.VISIT_CODE_DRG cod ON cod.VISITGUID = j.VISITGUID
WHERE j.JOURNEY_ID IN {}
'''.format(jids)
journey_information_df = pd.read_sql(query2,con)

In [9]:
# journey_information_df

In [41]:
notadmitted_df = sepsis_positive_df.loc[sepsis_positive_df.DISCHARGEDISPOSITION.isin(['ED01 Discharged Home','ED04 Did not wait', 'ED05 Left before treatment complete','ED08 Nursing Home'])].copy()


In [42]:
not_admitted_jids = notadmitted_df.JOURNEY_ID.unique()

In [43]:
jid_temp = not_admitted_jids[0]
jid_temp

'89113501634741464539-109'

In [44]:
journey_information_df.TYPECODE.unique()

array(['Inpatient', 'Emergency', 'Waiting List-IP'], dtype=object)

In [45]:
def timeDiff(jid, second_dtm,df):
    """
    Calculates the datetime difference between two datetimes  in days. 
    In this case the end date corresponds to ED discharge so if NAT 
    sets to zero as by definition goes straight to ward. 
    
    Parameters
    ----------
    first_dtm: datetime 
        datetime of first event
        
    second_dtm: datetime 
        datetime of second event
    
    Returns
    -------
    diff: float
        time difference measured in days 
    """
    # print(first_dtm,second_dtm)
    
    first_dtm = df.loc[df.JOURNEY_ID==jid].ADMITDTM.min()


    if pd.isna(second_dtm) == False:
        if pd.isna(first_dtm) == False:
            diff = (second_dtm - first_dtm)/np.timedelta64(1, 'm')
        else: 
            diff = None
    else: 
        diff = None

    
    return pd.to_datetime('1970-06-01 12:00') + pd.DateOffset(minutes=diff)

In [46]:
diff = 3
pd.to_datetime('1970-06-01') + pd.DateOffset(minutes=diff)

Timestamp('1970-06-01 00:03:00')

In [47]:
journey_df = journey_information_df.loc[journey_information_df.JOURNEY_ID.isin(not_admitted_jids)].copy()

In [48]:
journey_df['Admit Time (Days)'] = journey_df.apply(lambda row: timeDiff(row['JOURNEY_ID'],row['ADMITDTM'],journey_df), axis=1)
journey_df['Discharge Time (Days)'] = journey_df.apply(lambda row: timeDiff(row['JOURNEY_ID'],row['DISCHARGEDTM'],journey_df), axis=1)

In [49]:
journey_df.head()

,JOURNEY_ID,JOURNEY_ID_OLD,CLIENTGUID,SAUHI,RANK,EPISODE_ID,VISITGUID,CHARTGUID,ADMITDTM,DISCHARGEDTM,DISCHARGEDISPOSITION,PRE_VISITGUID,PRE_CHARTGUID,PRE_ADMITDTM,PRE_DISCHARGEDTM,PRE_DISCHARGEDISPOSITION,DISCHARGELOCATION,VISITIDCODE,AGEONADMIT,DOB,AGE,SUBURB,POSTCODE,HOMELHN,MH_CATCHMENT,SEIFA_RELATIVE_DISADVANTAGE,IRSD_SCORE,IRSD_DECILE,IRSAD_SCORE,IRSAD_DECILE,IER_SCORE,IER_DECILE,IEO_SCORE,IEO_DECILE,IRSD_GROUP,USUALACCOMODATIONTYPE,HOSPITAL,HOSPITALNAME,METROCOUNTRYFLAG,TYPECODE,CARELEVELCODE,CURRENTLOCATIONGUID,CURRENTLOCATION,WARD,WARD_DESC,WARD_TYPE,WARD_CAPACITY,REHAB_WARD_TYPE,REHAB_HITH_FLAG,ADMISSIONREQUESTDATETIME,GROUPCODE,SERVICEDESCRIPTION,SUBGROUPCODE,SUBGROUP_DESCRIPTION,CLINICCODE,CLINICNAME,ADMITSOURCE,ADMISSIONCATEGORY,EPISODEOFCARE,PATIENTCATEGORY,ADMISSIONTYPE,ADMISSIONELECTION,SOURCEOFREFERRAL,LOSNIGHTS,LOSHOUR,LOS_GROUPS,MORTALITY_INDICATOR,SAH_DIAGNOSIS,DIAGNOSISCODE,ED_CODE_2,ED_CODE_3,DIAGNOSIS_CODE_DESC,ALLDIAGNOSIS,ALLCHARDIAGNOSIS,ALLDIAGNOSISHOSPITAL,ALLCHARDIAGNOSISHOSPITAL,PROCEDURECODE,PROCEDURE_CODE_DESC,ALLPROCEDURES,LEGALSTATUSCODE,LEGALSTATUSDESCRIPTION,DRG,MDC,ECCS,NWAU,TILESEPISODEOFCARE,TILESDISCHARGEDISPOSITION,ICUVENTHOURS,TOTALMVHOURS,DICTIONARYREASONS,MODE_OF_ARRIVAL,MODE_OF_ARRIVAL_TYPE,TRIAGE_CATEGORY,SAAS_ORG_EVENT_NUMBER,SAAS_ORG_DISPATCH_NUMBER,ED_ORG_EVENT_NUMBER,ED_ORG_DISPATCH_NUMBER,SAAS_INCIDENT_PRIORITY,DESTINATIONARRIVALDATETIME,TRIAGEASSESSMENTDESCRIPTION,TRIAGEASSESSMENTSUBDESCRIPTION,LOCATIONCOMMONPLACENAME,LOCATIONPRIMARYCOMMONPLACENAME,LOCATIONCOMMONPLACETYPE,DESTINATIONCOMMONPLACENAME,DESTINATIONPRIMARYCOMMONPLACENAME,DESTINATIONCOMMONPLACETYPE,DISPOSITIONTYPECODE,DISPOSITIONTYPEDESCRIPTION,DESTINATIONFLAG,PATIENTHANDOVERDATETIME,AROC_RECORD_TYPE,AROC_NOTE_FLAG,MH_WARD_TYPE,TREATED_IN_MH_WARD,DEMENTIA_FLAG,MENTAL_HEALTH_ECT_TREATMENT_FLAG,MENTAL_HEALTH_ECT_JOURNEY_FLAG,JOURNEY_ACCURACY,JOURNEY_HOSPITAL_TYPE,FREQUENT_VISITOR_FLAG,MENTAL_HEALTH_FLAG_BYDIAGNOSIS,MENTAL_HEALTH_FLAG_BYJOURNEY,CLOZAPINE_FLAG_BYJOURNEY,MENTAL_HEALTH_FLAG_BYJOURNEY_MHWARD,REHAB_FLAG_BYJOURNEY,EDI_REHAB_AGECARE_FLAG_BYJOURNEY,MAINTENANCE_FLAG_BYVISIT,REHAB_FLAG_BYVISIT,GEM_FLAG_BYVISIT,ED_24HRS_IP_ADMISSION_FLAG,PATIENT_ED_24HRS_IP_ADMISSION_FLAG,CHRONIC_DIAGNOSIS,ONSITE_WARD_FLAG,EDI_REHAB_AGECARE_FLAG_BYVISIT,DRGCODE,Admit Time (Days),Discharge Time (Days)
147,22922801634741464539-109,1211265 - 9,21142900200,100018840874,3,9,2729587700270,1917348700170,2024-02-02 05:44:00+00:00,2024-02-20 04:30:00+00:00,IP Hosp in the Home,2.728996e+12,1.917349e+12,2024-02-02 03:16:00+00:00,2024-02-02 06:41:00+00:00,ED02 Admit as Inpatient,None,115467961,29,1995-02-15,30,SALISBURY NORTH,5108,NALHN,Northern Adelaide LHN,847.0,834,1,842,1,907,1,851,1,Low,Other Supported Accommodation,LMH,Lyell McEwin Hospital,M,Inpatient,Surgical,3336500610,LMH-Ward 2B-01_02-01,L2B,None,None,NaN,ACUTE,0.0,NaT,LMH-LCOLO,LMH IP Colorectal / COLORECTAL,LMH-LCOLO,LMH IP Colorectal,LCOLO,IP Colorectal,IP Casualty-Emergency,Emergency,Acute,Overnight Stay,Ordinary,Public,IP Casualty-Emergency,18,431,15-20 days,Didnt Die,Perforation of intestine (nontraumatic) :Dx,K255,None,None,Chronic or unsp gastric ulcer w perf,K25 K65 R57 K59 F20 E86 E88 A41 J90 J98 D64 N7...,K255 K6529 R572 K590 F209 E86 E880 A418 J90 J9...,J18 Y95 T81 Y83 R40 Y45 Y92,J189 Y95 T814 Y8334 R400 Y450 Y9224,3203000,Rectosigmoidectomy w stoma formation,3203000 9251420 1388201 9251529 9220900 920620...,3,Voluntary,G01A,006,7,10.0488,Acute,IP Hosp in the Home,None,87,None,None,Non Ambulance,None,None,None,None,None,None,NaT,None,None,None,None,None,None,None,None,None,None,None,NaT,None,Not Applicable,Other,N,0,N,N,Accurate Journey,All Metro,Y,Y,Y,N,N,N,N,N,N,N,N,Y,Not Chronic,Y,N,G01A,1970-06-01 19:54:00,1970-06-19 18:40:00
297,76229301634741464539-114,1724934 - 14,74449400200,100015335852,2,14,2690844400270,1895276200170,2024-01-07 06:16:00+00:00,2024-01-07 15:46:00+00:00,ED02 Admit as Inpatient,2.690780e+12,1.895224e+12,2024-01-07 02:42:00+00:00,2024-01-07 04:

In [19]:
# jid = 

In [50]:
jid = '21049601634741464539-106'
TimeFirst = journey_df.loc[journey_df.JOURNEY_ID==jid].ADMITDTM.min()
TimeFirst

NaT

In [53]:
colors = {'Emergency':'#32C0D2','Acute':'#106EA0','Rehabilitation':'#E0B165','Hospital at Home - Rehab at Home':'#00969E','Maintenance Care':'#963C4C','Palliative Care':'#744EC2'}


colors = {'Emergency':'#32C0D2','Acute':'#106EA0','Rehabilitation':'#E0B165','Emergency':'#00969E','Maintenance Care':'#963C4C','Palliative Care':'#744EC2'}



colors = {'Emergency': '#963C4C', 'Inpatient':'#32C0D2'}

color_dict = {0: to_rgba('#32C0D2', 1),
                1: to_rgba("#E0B165", 1),}

plt.style.use('./CEIH.mplstyle')

fig = px.timeline(journey_df,x_start='Admit Time (Days)',x_end = 'Discharge Time (Days)', color='TYPECODE',y='JOURNEY_ID',template='simple_white',
                  title='Patient Timelines - "Not admitted 2024"',color_discrete_map=colors,range_x=['1970-06-01 11:00','1970-06-04'],text='HOSPITAL',
                  labels={'JOURNEY_ID':'Journey Identifier'}) 



# fig = px.timeline(journey_df,x_start='ADMITDTM',x_end = 'DISCHARGEDTM', color='TYPECODE',color_discrete_map=colors,template='simple_white',
#                   title='Patient Timelines',
#                   labels = {'CARELEVELCODE':'Care Level Code','EPISODEOFCARE':'Episode of Care','GroupType':'Clinical Group','ADMITDTM':'Admit Date','DISCHARGEDTM':'Discharge Date'}) 
#                      #,template="plotly_white")
# "#32C0D2","#106EA0","#E0B165","#00969E","#963C4C","#744EC2","#D9B300","#D64550"
# fig.update(color=colors)
# fig.update_layout(
#     plot_bgcolor='white'
# )
fig.update_layout(legend_title_text='Admission Type')
fig.update_yaxes(autorange="reversed")
fig.update_yaxes()
fig.update_yaxes()
fig.update_traces(insidetextanchor='start')

fig.show()
fig.write_image("patient_timelines_not_admitted_2024.png",width=1200, height=1200, scale=10)

In [22]:
journey_df.columns

Index(['JOURNEY_ID', 'JOURNEY_ID_OLD', 'CLIENTGUID', 'SAUHI', 'RANK',
       'EPISODE_ID', 'VISITGUID', 'CHARTGUID', 'ADMITDTM', 'DISCHARGEDTM',
       ...
       'REHAB_FLAG_BYVISIT', 'GEM_FLAG_BYVISIT', 'ED_24HRS_IP_ADMISSION_FLAG',
       'PATIENT_ED_24HRS_IP_ADMISSION_FLAG', 'CHRONIC_DIAGNOSIS',
       'ONSITE_WARD_FLAG', 'EDI_REHAB_AGECARE_FLAG_BYVISIT', 'DRGCODE',
       'Admit Time (Days)', 'Discharge Time (Days)'],
      dtype='object', length=138)

In [23]:
col = 'ADMITDTM'
journey_df[col] = (journey_df[col]).dt.tz_convert('Australia/Adelaide')

col = 'DISCHARGEDTM'
journey_df[col] = (journey_df[col]).dt.tz_convert('Australia/Adelaide')


In [24]:
journey_df.sort_values(['JOURNEY_ID','ADMITDTM'])[['JOURNEY_ID','VISITGUID','CLIENTGUID','TYPECODE','ADMITDTM','DISCHARGEDTM','DISCHARGEDISPOSITION','HOSPITAL','RANK']]

,JOURNEY_ID,VISITGUID,CLIENTGUID,TYPECODE,ADMITDTM,DISCHARGEDTM,DISCHARGEDISPOSITION,HOSPITAL,RANK
2505,12541001634741464539-111,2470983500270,10761100200,Emergency,2023-08-03 13:28:00+09:30,2023-08-03 16:10:00+09:30,ED05 Left before treatment complete,LMH,1
3893,12541001634741464539-111,2470404600270,10761100200,Emergency,2023-08-03 20:09:00+09:30,2023-08-04 18:22:00+09:30,ED02 Admit as Inpatient,LMH,2
3164,12541001634741464539-111,2473467800270,10761100200,Inpatient,2023-08-03 21:00:00+09:30,2023-08-08 17:14:00+09:30,IP Self Discharge - Signed,LMH,3
7195,13596501634741464539-111,2263623900270,11816600200,Emergency,2023-03-04 03:37:00+10:30,2023-03-04 09:10:00+10:30,ED01 Discharged Home,NHS,1
3775,13596501634741464539-111,2263676000270,11816600200,Inpatient,2023-03-04 09:43:00+10:30,2023-03-16 18:50:00+10:30,IP Home,FMC,2
1523,21049601634741464539-106,2391008000270,19269700200,Emergency,2023-06-07 12:56:00+09:30,2023-06-07 18:53:00+09:30,ED04 Did not wait,FMC,1
813,21049601634741464539-106,2391976300270,19269700200,Emergency,2023-06-08 00:18:00+09:30,2023-06-08 13:44:00+09:30,ED02 Admit as Inpatient,FMC,2
4736,21049601634741464539-106,2393018600270,19269700200,Inpatient,2023-06-08 06:50:00+09:30,2023-06-14 11:26:00+09:30,IP Home,FMC,3
7225,24412201454741464540-105,2460371300270,226323000200,Emergency,2023-07-27 10:49:00+09:30,2023-07-27 17:03:00+09:30,ED01 Discharged Home,RAH,1
5606,24412201454741464540-105,2461593900270,226323000200,Emergency,2023-07-27 19:47:00+09:30,2023-07-28 18:57:00+09:30,ED02 Admit as Inpatient,RAH,2


In [25]:
journey_df.JOURNEY_ID.unique()

array(['35214903434741464538-104', '21049601634741464539-106',
       '51159701634741464539-152', '53420101634741464539-116',
       '82382401634741464539-102', '53539601634741464539-105',
       '63499001634741464539-105', '40560201634741464539-105',
       '12541001634741464539-111', '47100903434741464538-101',
       '13596501634741464539-111', '24412201454741464540-105'],
      dtype=object)

In [26]:
notadmitted_df.VISITGUID

170528    2460371300270
177480    2313732600270
228668    2444545700270
259365    2355895800270
306244    2677279900270
341232    2318902200270
449990    2263623900270
545067    2375177100270
583310    2391008000270
599703    2470983500270
605773    2569801500270
608916    2191688600270
Name: VISITGUID, dtype: Int64